# MA 100.2 Project 2: Forward Rate Agreements

### Instructor: Juan Carlo F. Mallari (jmallari@ateneo.edu)

MATH 100.2: Topics in Financial Mathematics II \
First Semester, S.Y. 2022-2023 \
Ateneo de Manila University

Submitted by:
1. BELTRAN, Neil John
2. DONAIRE, Aengus Martin
3. VICTORIA, Maj

People I asked help from: None

Websites I used for completing this Project:
1. https://numpy.org/doc/stable/reference/generated/numpy.cov.html

In [1]:
import numpy as np # Numerical Computing
import pandas as pd # Data wrangling
import matplotlib.pyplot as plt #Plotting
import datetime as dt

from statistics import NormalDist # statistical analysis
from tabulate import tabulate #Tables
from dateutil.relativedelta import relativedelta

# PART 1: Functional Implementation

In [2]:
def linear_interp(t,x,y):
    
    """
    Linearly interpolates the zero rate for tenor t 
    using the equation of line determined by the two available points 
    (x_1,y_1) and (x_2,y_2) whose tenors x_1 and x_2 are nearest to t
    
    -------
    Inputs:
    t = tenor of the unknown zero rate 
    x = list [x_1,x_2]
    y = list [y_1,y_2]
    
    Return
    -------
    float (The y value of t in a line with slope m and passes thru (x[0],y[0]))
    """
    
    m = (y[1]-y[0])/(x[1]-x[0])
    
    y = y[0] + m*(t-x[0])
    
    return y

In [3]:
def convert_freq_rate(R_n,n,m,freq):
    
    """this function converts rates from n-compounding with rate R_n to m-compounding with rate R_m,
    and from n-compounding with rate R_n to continuous rates with rate R_c"""
    
    if freq =='exp':
        R= n*np.log(1+ R_n/n)
        
    else:
        R= m*((1+R_n/n)**(n/m) -1)
        
    return R

In [4]:
def forward_rate(
    t, T_1, T_2, r_1, r_2, m 
):
    """Returns the forward rate observed at time t (0<=t)
    (where T_1 < T_2 and their values are determined at time 0)
    
       This function assumes that the zero rates r_1 and r_2 and the forward rate 
       have the same compounding frequency. 
    
    Parameters
    ----------
    t (in years)
    T_1: start of forward period as observed at time t=0 (in years)
    T_2: end of forward period as observed at time t=0 (in years)
    r_1: zero rate observed at time t with tenor T_1-t years
    r_2: zero rate observed at time t with tenor T_2-t years
    m:  compounding frequency of r_1, r_2 and the forward rate 
        assumes value in this list: ['simple', 'exp', 1, 2, 4, 12, 52, 365]
    
    Returns
    -------
    float (4 decimal places)
        forward rate observed at time t (0<=t)
    """
    #for simple compounding
    if m == 'simple':
        DF_1 = (1+(r_1*(T_1-t)))**(-1)
        DF_2 = (1+(r_2*(T_2-t)))**(-1)
        forward_rate = (DF_1/DF_2 -1)/(T_2 - T_1)
    
    #for continuous compounding
    elif m== 'exp':
        forward_rate = (r_2*(T_2-t)-r_1*(T_1-t))/(T_2 - T_1)
    
    #for m-compounding
    else:
        DF_1 = (1 + r_1/m)**(-m*(T_1-t))
        DF_2 = (1 + r_2/m)**(-m*(T_2-t))
        forward_rate = m*((DF_1/DF_2)**(1/(m*(T_2-T_1)))-1)

    return forward_rate

In [5]:
def fra_value(
    t, T_1, T_2, r_fixed, r_float, r_t, m, L, payer 
):
    """Returns the value of the forward rate agreement (FRA) at time t (0<t<=T_1) to the specified payer.
    
       At time t=T_1, the function returns the present value (at time t=T_1) of the FRA's cash settlement 
       from the perspective of the specified payer.
        
       This function assumes that the interest rates r_fixed, r_float, and r_t have the same compounding frequency. 
    """
    """
    Parameters
    ----------
    t (in years) 
    T_1: start of forward period as observed at time t=0 (in years)
    T_2: end of forward period as observed at time t=0 (in years)
    
    r_fixed: fixed interest rate of the FRA determined at time t=0
    r_float: floating interest rate of the FRA determined at time t
    r_t: zero rate observed at time t with tenor T_2-t years
    
    m: compounding frequency of r_1, r_2 and the forward rate 
        assumes value in this list: ['simple', 'exp', 1, 2, 4, 12, 52, 365]
    L: notional principal amount of the FRA
    payer: we are getting the value of the FRA from the perspective of the payer
        assumes value in this list: ['fixed', 'float']
    
    Returns
    -------
    float (2 decimal places)
        value of the forward rate agreement (FRA) at time t (0<t<=T_1) to the specified payer
    """
    
    if m == 'simple':
        t_2   = T_2 -t
        DFt_2 = (1+ r_t*t_2)**(-1)
        V_t   = L*(r_float- r_fixed)*(T_2-T_1)*DFt_2
        
        if payer == 'fixed':
            V_tp = V_t
        else:
            V_tp = -V_t
            
    elif m == 'exp':
        t_2   = T_2 -t
        DFt_2 = np.exp(-r_t*t_2)
        V_t   = L*(np.exp(r_float*(T_2-T_1))-np.exp(r_fixed*(T_2-T_1))) *DFt_2
        
        if payer == 'fixed':
            V_tp = V_t
        else:
            V_tp = -V_t
            
    else:
        t_2   = T_2 -t
        DFt_2 = (1 + r_t/m)**(-m*t_2)
        
        V_t   = L*((1+r_float/m)**(m*(T_2-T_1))-(1+r_fixed/m)**(m*(T_2-T_1)))*DFt_2
        
        if payer == 'fixed':
            V_tp = V_t
    
        else:
            V_tp = -V_t
    
    return round(V_tp,2)

# PART 2: Validation

#### QUESTION A

Given the following zero rates: 1-month: 1.75%, 3-month:2.5%, 6-month: 3.75%, 9-month: 4.25%, 1-year: 4.75%.z

In [6]:
#From the given, the rates are:

R1 = 1.75 /100
R3 = 2.5  /100
R6 = 3.75 /100
R9 = 4.25 /100
R12= 4.75 /100

In [7]:
#If the given zero rates are simple rates, calculate the 3-month forward rate 3 months from now.

a1 = forward_rate(
    t   = 3/12   ,
    T_1 = 6/12   ,
    T_2 = 9/12   ,
    r_1 = R3     ,
    r_2=  R6     ,
    m= 'simple' 
)
print(f"If zero rates are simple, the 3-month forward rate 3 months from now is {format(a1*100,',')}%.") 

If zero rates are simple, the 3-month forward rate 3 months from now is 4.9689440993788025%.


In [8]:
#If the given zero rates are compounded annually, calculate the 5-month forward rate 1 month from now.

a2 = forward_rate(
    t   = 1/12   ,
    T_1 = 2/12   ,
    T_2 = 7/12   ,
    r_1 = R1     ,
    r_2=  R6     ,
    m= 1 
)
print(f"Compounded annually, the 5-month forward rate 1 month from now is {format(a2*100,',')}%.") 

Compounded annually, the 5-month forward rate 1 month from now is 4.154692934071957%.


In [9]:
#If the given zero rates are compounded semi-annually, calculate the 6-month forward rate 6 months from now.

a3 = forward_rate(
    t   = 6/12     ,
    T_1 = 12/12    ,
    T_2 = 18/12    ,
    r_1 = R6       ,
    r_2=  R12      ,
    m= 2   
)
print(f"Compounded semi-annually, the 6-month forward rate 6 months from now is {format(a3*100,',')}%.") 

Compounded semi-annually, the 6-month forward rate 6 months from now is 5.7549079754601%.


In [10]:
#If the given zero rates are compounded monthly, calculate the 6-month forward rate 3 months from now.

a4 = forward_rate(
    t   = 3/12     ,
    T_1 = 6/12     ,
    T_2 = 12/12    ,
    r_1 = R3       ,
    r_2=  R9       ,
    m= 12
)
print(f"Compounded monthly, the 6-month forward rate 3 months from now is {format(a4*100,',')}%.") 

Compounded monthly, the 6-month forward rate 3 months from now is 5.1259548100610175%.


In [11]:
#If the given zero rates are continuously-compounded, calculate the 3-month forward rate 6 months from now.

a5 = forward_rate(
    t   = 3/12     ,
    T_1 = 9/12    ,
    T_2 = 12/12    ,
    r_1 = R6       ,
    r_2=  R9       ,
    m= 'exp'
)
print(f"If zero rates are continuously-compounded, the 3-month forward rate 6 months from now is {format(a5*100,',')}%.") 

If zero rates are continuously-compounded, the 3-month forward rate 6 months from now is 5.250000000000001%.


### Question B

Suppose that a $(3 \times 6)$ FRA is set today with notional 15 million pesos. The 3-month rate now with continuous compounding is 2.250% and the 6-month rate with continuous compounding is 3.175%.

In [12]:
#1. Determine the fixed rate for the FRA.

b1 = forward_rate(
    t   = 0          ,
    T_1 = 3/12       ,
    T_2 = 6/12       ,
    r_1 = 2.25/100   ,
    r_2=  3.175/100  ,
    m= 'exp' 
) 

print(f"The fixed rate for this FRA is {round(b1*100,2)}%.") 

The fixed rate for this FRA is 4.1%.


In [13]:
#2. After 1 month, the 2-month rate is 2.125% and the 5-month rate is 3.425%, both with continuous compounding.
#What is the value of the FRA to the fixed-rate payer? to the floating-rate payer?

rfloatb1 = forward_rate(
    t    = 1/12       ,
    T_1  = 3/12       ,
    T_2  = 6/12       ,
    r_1  = 2.125/100  ,
    r_2  =  3.425/100 ,
    m    = 'exp' 
) 

b2 =fra_value(
    t =1/12, T_1 =3/12, T_2 =6/12, 
    r_fixed =b1, 
    r_float =rfloatb1,
    r_t     =3.425/100,
    m       ='exp',
    L       =15000000, 
    payer   ='fixed' 
)

print(f"The value of the FRA to the fixed-rate payer is {format(b2,',')} while its value to the floating-rate payer is {format(-b2,',')}")

The value of the FRA to the fixed-rate payer is 7,160.37 while its value to the floating-rate payer is -7,160.37


In [14]:
#3. Suppose the 3-month rate at expiration date of the FRA is 3.915% with continuous compounding. 
#Which party pays the other party? How much is the settlement amount?

rfloatb2 = 3.915/100

b3 =fra_value(
    t       =3/12      ,
    T_1     =3/12      , 
    T_2     =6/12      , 
    r_fixed =b1        ,
    r_float =rfloatb2  ,
    r_t     =rfloatb2  ,
    m       ='exp'     , 
    L       =15000000  , 
    payer   ='fixed' 
)

if b1 > rfloatb2:
    print(f"The fixed-rate payer pays the floating-rate payer with the settlement amount of {format(abs(b3),',')}")

else:
    print(f"The float-rate payer pays the fixed-rate payer with the settlement amount of {format(abs(b3),',')}")

The fixed-rate payer pays the floating-rate payer with the settlement amount of 6,939.1


### Question C

A $(6 \times 12)$ FRA is set today with notional 20 million pesos. The 6-month zero rate with semi-annual compounding is 3.875% and the 1-year interest rate with semi-annual compounding is 4.125%.

In [15]:
#1. Determine the fixed rate for the FRA.

c1 = forward_rate(
    t   = 0           ,
    T_1 = 6/12        ,
    T_2 = 12/12       ,
    r_1 = 3.875/100   ,
    r_2 = 4.125/100   ,
    m   = 2 
) 

print(f"The fixed rate for this FRA is {round(c1*100,2)}%.") 

The fixed rate for this FRA is 4.38%.


In [16]:
#2. After 3 months, the 3-month zero rate with quarterly compounding is 3.225% and the 9-month zero rate with quarterly compounding is 3.725%. 
#What is the value of the FRA to the fixed-rate payer? to the floating-rate payer?

#Rates are coverted to semi-annual compounding
rt_semi3  = convert_freq_rate(R_n=0.03225,n=4,m=2,freq='notexp')
rt_semi9  = convert_freq_rate(R_n=0.03725,n=4,m=2,freq='notexp')

rfloatc1 = forward_rate(
    t   = 3/12        ,
    T_1 = 6/12        ,
    T_2 = 12/12       ,
    r_1 = rt_semi3    ,
    r_2 = rt_semi9    ,
    m   = 2 
)

c2 =fra_value(
    t       =3/12       ,
    T_1     =6/12       , 
    T_2     =12/12      , 
    r_fixed = c1        ,
    r_float = rfloatc1  ,
    r_t     = rt_semi9  ,
    m       = 2         ,
    L       = 20000000  , 
    payer   ='fixed' 
)

print(f"The value of the FRA to the fixed-rate payer is {format(c2,',')} while its value to the floating-rate payer is {format(-c2,',')}")

The value of the FRA to the fixed-rate payer is -36,989.07 while its value to the floating-rate payer is 36,989.07


In [17]:
#3. Suppose the 6-month zero rate with semi-annual compounding at expiration date of the FRA is 4.425%. 
#Which party pays the other party? How much is the settlement amount??

rfloatc3 = 4.425/100

c3 =fra_value(
    t       =6/12       ,
    T_1     =6/12       , 
    T_2     =12/12      , 
    r_fixed = c1        ,
    r_float = rfloatc3  ,
    r_t     = rfloatc3  ,
    m       = 2         ,
    L       = 20000000  , 
    payer   ='fixed' 
)

if c1 > rfloatc3:
    print(f"The fixed-rate payer pays the floating-rate payer with the settlement amount of {format(c3,',')}")

else:
    print(f"The floating-rate payer pays the fixed-rate payer with the settlement amount of {format(c3,',')}")

The floating-rate payer pays the fixed-rate payer with the settlement amount of 4,861.78


### Question D

A $(6 \times 18)$ FRA with notional 5 million pesos was set 2 months ago for a fixed simple rate of 6.725%. Suppose the simple zero rates now are as follows.

In [18]:
#1. Use linear interpolation to estimate the 4-month and 16-month zero rates.

R4_a  = linear_interp(t= 4/12 ,x=[0.25,0.5]   ,y=[0.04515,0.04815])
R16_a = linear_interp(t= 16/12,x=[12/12,24/12],y=[0.06235,0.06925])

print(f"The estimated 4-month and 16-month zero rates are {round(R4_a*100,4)}% and {round(R16_a*100,4)}%, respectively") 

The estimated 4-month and 16-month zero rates are 4.615% and 6.465%, respectively


In [19]:
#2. Determine the value of the FRA now to the fixed-rate payer.

d1 = forward_rate(
    t   = 0           ,
    T_1 = 4/12        ,
    T_2 = 16/12       ,
    r_1 = R4_a        ,
    r_2 = R16_a       ,
    m   = 'simple' 
)

d2 =fra_value(
    t       =0         ,
    T_1     =4/12      , 
    T_2     =16/12     , 
    r_fixed =6.725/100 ,
    r_float =d1        ,
    r_t     =R16_a     ,
    m       ='simple'  ,
    L       =5000000   , 
    payer   ='fixed'
)

print(f"The value of the FRA to the fixed-rate payer is {format(d2,',')}")

The value of the FRA to the fixed-rate payer is 11,479.35


In [20]:
### 3. After 2 months (from now), the simple zero rates are as follows. What is the value of the FRA to the floating-rate payer?

R2  = linear_interp(t= 2/12,x=[1/12,3/12],y=[4.125/100,4.405/100])
R4  = linear_interp(t= 4/12,x=[3/12,6/12],y=[4.405/100,5.015/100])
R14 = linear_interp(t= 14/12,x=[12/12,24/12],y=[5.925/100,6.215/100])
R16 = linear_interp(t= 16/12,x=[12/12,24/12],y=[5.925/100,6.215/100])

rfl = forward_rate(
    t   = 2/12        ,
    T_1 = 4/12        ,
    T_2 = 16/12       ,
    r_1 = R2          ,
    r_2 = R14         ,
    m   = 'simple' 
)

d4 =fra_value (
    t       =2/12        ,
    T_1     =4/12        , 
    T_2     =16/12       , 
    r_fixed =6.725/100   ,
    r_float =rfl         ,
    r_t     =R14         ,
    m       ='simple'    ,
    L       =5000000     , 
    payer   ='float'
)

print(f"The value of the FRA to the floating-rate payer is {format(d4,',')}")

The value of the FRA to the floating-rate payer is 23,890.82


In [23]:
#4. At expiration date, the simple zero rates are as follows. Which party pays the other party? How much is the settlement amount?

rfloatd = 6.075/100
rfx = 6.725/100

R4d  = linear_interp(t= 4/12,x=[3/12,6/12],y=[4.455/100,4.915/100])
R16d = linear_interp(t= 16/12,x=[12/12,24/12],y=[6.075/100,6.565/100])

d5 =fra_value (
    t       =4/12        ,
    T_1     =4/12        , 
    T_2     =16/12       , 
    r_fixed =6.725/100   ,
    r_float =rfloatd     ,
    r_t     =rfloatd     ,
    m       ='simple'    ,
    L       =5000000     , 
    payer   ='float'
)

if rfx > rfloatd:
    print(f"The fixed-rate payer pays the floating-rate payer with the settlement amount of {format(d5,',')}")

else:
    print(f"The floating-rate payer pays the fixed-rate payer with the settlement amount of {format(d5,',')}")

The fixed-rate payer pays the floating-rate payer with the settlement amount of 30,638.7
